<a href="https://colab.research.google.com/github/mehdimerbah/COVID19_fake_news_detection/blob/main/models/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Library and Data Import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM, Dense
import json

In [ ]:
dataset = pd.read_csv('https://raw.githubusercontent.com/mehdimerbah/COVID19_fake_news_detection/main/preprocessing/whole_processed_data.csv')

# One-Hot Data Representation
TBD:    
- [ ] Add one hot representation explanation   
- [ ] Explain embedding     
- [ ] Explain padding for the matrix    

In [ ]:
oh_representation = [one_hot(terms, 5000) for terms in dataset['tweet']]

In [ ]:
sentence_len = 400
embedded_data = pad_sequences(oh_representation, padding='pre', maxlen= sentence_len)

In [ ]:
embedded_data[:5]

array([[   0,    0,    0, ..., 4424, 2850, 4175],
       [   0,    0,    0, ..., 2683, 1996, 4697],
       [   0,    0,    0, ..., 3009, 3460, 3711],
       [   0,    0,    0, ..., 1152, 2158, 2635],
       [   0,    0,    0, ..., 3972, 1490, 4541]], dtype=int32)

In [ ]:
# For easier manipulation I will transform the 2D array into a pandas DataFrame
embedded_DF = pd.DataFrame(embedded_data)
# Adding one column for the class labels to the DF
embedded_DF['label'] = dataset['label']
# Converting the lables from (real/fake) strings to (1/0)
embedded_DF['label'] = embedded_DF.label.eq('real').mul(1)

# Data Splitting
training_set = embedded_DF.iloc[:6420]
validation_set = embedded_DF.iloc[6420:8560]
testing_set = embedded_DF.iloc[8560:]
embedded_DF.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,label
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2587,648,4712,4268,4697,3023,3294,4801,3994,2201,4111,3564,707,4801,4075,2460,4207,4424,2850,4175,1
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4192,2683,1944,4697,3564,2597,2538,2255,1726,4192,2683,1996,4697,1
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,657,3332,1352,3648,1575,3676,751,1963,2466,3009,3460,3711,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3621,4703,2266,1490,449,2480,291,882,4064,2235,2127,2437,4742,1849,1152,2158,2635,1
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4338,4192,858,1546,1209,3994,1924,2132,4235,2542,69,4175,3845,1667,4192,3186,4235,2542,69,658,4458,1063,3818,4362,328,4511,1982,3972,1490,4541,1


# Building the Model

TBD: 
- [ ] Elaborate on sequential models     
- [ ] Why did we choose 5000 word embedding ?
- [ ] Sigmoid vs tanh ?    
- [ ] Highlight optimizer and explain that the other metrics are added to results    
- [ ] Describe the model (params, layers, activation functions)

In [ ]:
# initializing model
model = Sequential()
# adding embedding layer
model.add(Embedding(5000, 100, input_length=sentence_len))
model.add(LSTM(150))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 400, 100)          500000    
                                                                 
 lstm_1 (LSTM)               (None, 150)               150600    
                                                                 
 dense_1 (Dense)             (None, 1)                 151       
                                                                 
Total params: 650,751
Trainable params: 650,751
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_history = model.fit(training_set.iloc[:,:400], training_set.iloc[:,400], validation_data=(validation_set.iloc[:,:400], validation_set.iloc[:,400] ), epochs=10, batch_size=64)

Epoch 1/10
101/101 [==============================] - 111s 1s/step - loss: 0.3815 - accuracy: 0.8282 - val_loss: 0.2182 - val_accuracy: 0.9103
Epoch 2/10
101/101 [==============================] - 108s 1s/step - loss: 0.1472 - accuracy: 0.9456 - val_loss: 0.2416 - val_accuracy: 0.9000
Epoch 3/10
101/101 [==============================] - 108s 1s/step - loss: 0.1196 - accuracy: 0.9687 - val_loss: 0.2332 - val_accuracy: 0.9145
Epoch 4/10
101/101 [==============================] - 109s 1s/step - loss: 0.0455 - accuracy: 0.9855 - val_loss: 0.3007 - val_accuracy: 0.9121
Epoch 5/10
101/101 [==============================] - 109s 1s/step - loss: 0.0326 - accuracy: 0.9905 - val_loss: 0.3161 - val_accuracy: 0.9140
Epoch 6/10
101/101 [==============================] - 109s 1s/step - loss: 0.0148 - accuracy: 0.9967 - val_loss: 0.3914 - val_accuracy: 0.9042
Epoch 7/10
101/101 [==============================] - 110s 1s/step - loss: 0.0142 - accuracy: 0.9963 - val_loss: 0.4209 - val_accuracy: 0.8930

In [ ]:
# Now we can use the model to make some predictions.
## The predictions values are squeezed between 0 and 1 by the sigmoid activation 
## We need to convert them to 1 or 0, to fit the classes
predictions = model.predict(validation_set.iloc[:,:400])
predictions[:5]

array([[1.9494952e-05],
       [9.9897122e-01],
       [3.5098195e-04],
       [7.4055833e-06],
       [9.9991858e-01]], dtype=float32)

In [ ]:
# To convert the values, we simply compare to 0.5 and round it
predictions = (predictions > 0.5).astype("int32")
predictions

array([[0],
       [1],
       [0],
       ...,
       [0],
       [0],
       [1]], dtype=int32)

# Results

In [ ]:
print(classification_report(validation_set['label'], predictions))

              precision    recall  f1-score   support

           0       0.89      0.89      0.89      1020
           1       0.90      0.90      0.90      1120

    accuracy                           0.90      2140
   macro avg       0.90      0.90      0.90      2140
weighted avg       0.90      0.90      0.90      2140



In [ ]:
round(accuracy_score(validation_set['label'], predictions), 5)

0.89907

In [ ]:
def get_metrics(predicted,true):
    metrics = dict()
    metrics['accuracy'] = round(accuracy_score(predicted, true), 5)
    metrics['precision'] = round(precision_score(predicted, true, average = 'weighted'), 5)
    metrics['recall'] = round(recall_score(predicted, true, average = 'weighted'), 5)
    metrics['f1'] = round(f1_score(predicted, true, average = 'weighted'), 5)
    
    return metrics
    

In [ ]:
metrics = get_metrics(predictions, validation_set['label'])
print(metrics)

{'accuracy': 0.89907, 'precision': 0.89907, 'recall': 0.89907, 'f1': 0.89907}


In [ ]:
with open("LSTM_results.json", "w") as output:
    json.dump(metrics, output)